# Initial Data Exploration of Seoul Sensor Data

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(style="ticks", color_codes=True)
%matplotlib inline

In [ ]:
path_raw = '../data/raw/'
filename = 'Measurement_summary.csv'

In [ ]:
df = pd.read_csv(path_raw+filename)

In [ ]:
# rename some columns so that we can spend less time typing

df.rename(columns={"Measurement date": "date",
                   "Station code": "station",
                   "Address": "address",
                   "Latitude": "lat",
                   "Longitude": "lon"}, inplace = True)

In [ ]:
# Let's drop duplicate rows before anything else

shape_o = df.shape
df.drop_duplicates(keep='first', inplace=True)
shape_d = df.shape

if shape_o == shape_d:
    print("The original df contains {} rows and {} columns.".format(shape_o[0], shape_o[1]),
          "\nNo duplicate rows need to be dropped.")
else:
    print("The original df contains {} rows and {} columns.".format(shape_o[0], shape_o[1]),
          "\nAfter dropping duplicate rows, the df contains {} rows and {} columns.".format(shape_d[0],
                                                                                            shape_d[1]))

In [ ]:
# ensure "date" column has the proper datetime format and set it to index

df['date'] = pd.to_datetime(df['date'])
df.set_index('date', inplace=True)
df.to_pickle('../data/interim/summary.pkl')

df.describe()

In [ ]:
# How many stations of data do we have?

num_stations = len(df.station.unique())
print("The number of unique stations in this data set is {}.".format(num_stations))

In [ ]:
# Get a bounding box to visualize the sensor spread in Seoul

BBox = ((df.lon.min(), df.lon.max(),      
         df.lat.min(), df.lat.max()))

In [ ]:
# map sensors on top of Seoul map image from opestreetmap.org

seoul_map = plt.imread('images/Seoul_exact.png')

fig, ax = plt.subplots(figsize = (8,7))
ax.scatter(df.lon, df.lat, zorder=1, alpha= 0.2, c='m', s=10)
ax.set_title('Plotting Sensor Locations on Seoul Map')
ax.set_xlim(BBox[0],BBox[1])
ax.set_ylim(BBox[2],BBox[3])
ax.imshow(seoul_map, zorder=0, extent = BBox, aspect= 'equal')
plt.show()

In [ ]:
# do we have any nulls or nans?

df.isnull().values.any()

In [ ]:
# How many hours per station are missing and which hours?

date_rng = pd.date_range(start='1/1/2017', end='1/1/2020', freq='H', closed='left')
print("The count of datetimes should be {}.".format(len(date_rng)))

In [ ]:
df_hrs = df[['station', 'lat']].groupby('station').count()
df_hrs.rename(columns={"lat": "hrs_available"}, inplace=True)
df_hrs['hrs_missing'] = df_hrs.apply(lambda row: row.hrs_available - len(date_rng), axis=1)

In [ ]:
plt.figure(figsize=(15,7))
for i in range(101,126):
    df_del = df[df.station == i]
    df_del = df_del.reindex(date_rng, fill_value=(-1-(i-100)))
    df_del['hrs_missing'] = df_del.station.where(df_del.station == (-1-(i-100)))
    plt.plot(df_del.index, df_del.hrs_missing, label="Station {} missing {} hrs".format(i,-df_hrs.hrs_missing[i]))
plt.legend(bbox_to_anchor=(1.02, 0, 1.02, 1), loc='lower left', ncol=1, borderaxespad=0.)
plt.show()
    

### Let's look at some pairplots to see distributions and relationships

In [ ]:
# first, load masks

df_instruments_0 = pd.read_pickle('../data/interim/instrument_mask_0.pkl')
df_instruments_1 = pd.read_pickle('../data/interim/instrument_mask_1.pkl')
df_instruments_2 = pd.read_pickle('../data/interim/instrument_mask_2.pkl')
df_instruments_4 = pd.read_pickle('../data/interim/instrument_mask_4.pkl')
df_instruments_8 = pd.read_pickle('../data/interim/instrument_mask_8.pkl')
df_instruments_9 = pd.read_pickle('../data/interim/instrument_mask_9.pkl')


In [ ]:
# take subset of pollutant columns from summary dataframe to standardize
# change the index so it is compatible with the masks

df_to_std = df[['SO2', 'NO2', 'O3','CO','PM10','PM2.5']].copy()

df_to_std['idx'] = df_instruments_0.index
df_to_std.set_index('idx', inplace=True)


In [ ]:
# mask dataframes

df_to_std_0 = df_to_std.copy().where(df_instruments_0)
df_to_std_1 = df_to_std.copy().where(df_instruments_1)
df_to_std_2 = df_to_std.copy().where(df_instruments_2)
df_to_std_4 = df_to_std.copy().where(df_instruments_4)
df_to_std_8 = df_to_std.copy().where(df_instruments_8)
df_to_std_9 = df_to_std.copy().where(df_instruments_9)


In [ ]:
# normalize dataframes

df_std_all = (df_to_std - df_to_std.mean())/df_to_std.std()
df_std_0 = (df_to_std_0 - df_to_std_0.mean())/df_to_std_0.std()
df_std_1 = (df_to_std_1 - df_to_std_1.mean())/df_to_std_1.std()
df_std_2 = (df_to_std_2 - df_to_std_2.mean())/df_to_std_2.std()
df_std_4 = (df_to_std_4 - df_to_std_4.mean())/df_to_std_4.std()
df_std_8 = (df_to_std_8 - df_to_std_8.mean())/df_to_std_8.std()
df_std_9 = (df_to_std_9 - df_to_std_9.mean())/df_to_std_9.std()

In [ ]:
# pickle dataframes

df_to_std.to_pickle('../data/interim/summary_all.pkl')
df_to_std_0.to_pickle('../data/interim/summary_0.pkl')
df_to_std_1.to_pickle('../data/interim/summary_1.pkl')
df_to_std_2.to_pickle('../data/interim/summary_2.pkl')
df_to_std_4.to_pickle('../data/interim/summary_4.pkl')
df_to_std_8.to_pickle('../data/interim/summary_8.pkl')
df_to_std_9.to_pickle('../data/interim/summary_9.pkl')

In [ ]:
# pickle dataframes

df_std_all.to_pickle('../data/interim/summary_std_all.pkl')
df_std_0.to_pickle('../data/interim/summary_std_0.pkl')
df_std_1.to_pickle('../data/interim/summary_std_1.pkl')
df_std_2.to_pickle('../data/interim/summary_std_2.pkl')
df_std_4.to_pickle('../data/interim/summary_std_4.pkl')
df_std_8.to_pickle('../data/interim/summary_std_8.pkl')
df_std_9.to_pickle('../data/interim/summary_std_9.pkl')

In [ ]:
# normalized values with all instrument statuses

sns.pairplot(df_std_all)


In [ ]:
# normalized values with instrument statuses of normal

sns.pairplot(df_std_0)
